In [ ]:
import pandas as pd
from scipy.stats import uniform, geom, loguniform, randint, expon
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline as Pipeline
from sklearn.datasets import load_iris, load_boston, load_diabetes, load_digits, load_linnerud, load_wine, load_breast_cancer

In [ ]:
TEST_SIZE = 0.1
RANDOM_STATE = 10

data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

X = df.drop(['target'], axis=1)
y = df['target'].astype(float)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y) # 

In [ ]:
N_JOBS = 1
K_FOLDS = 10
N_ITER_NO_CHANGE = 100
SCORING_METRIC = 'roc_auc'

# Gradient Boosting Parameters
hyper_parameters = {
        'model__n_estimators': [50, 150, 300],
        'model__learning_rate': [0.1, 0.03, 0.001],
        'model__max_depth': [3, 5, 8],
        'model__max_features': ['sqrt'], 
        'model__min_samples_split': [2],
        'model__n_iter_no_change': [N_ITER_NO_CHANGE], # Note single elements should still be in a list
        'model__random_state': [RANDOM_STATE],
        'model__min_samples_leaf': [2, 10]
}

# Make pipeline - Add steps if needed, i.e. feature selection
pipe = Pipeline(steps=[
    ('model', GradientBoostingClassifier())
])

# 'n_iter' no longer needed as all combinations are checked
search_space = GridSearchCV(
    pipe, hyper_parameters, 
    cv=K_FOLDS, scoring=SCORING_METRIC, n_jobs = N_JOBS, 
    return_train_score=True, verbose = 1
)
search_space.fit(X_train, y_train) 

y_pred = search_space.best_estimator_.predict(X_train)
y_pred_prob = search_space.best_estimator_.predict_proba(X_train)[:,1]
y_pred_test = search_space.best_estimator_.predict_proba(X_test)[:,1]

print( 
    'Best Training Score: ', search_space.cv_results_['mean_train_score'][search_space.best_index_], 
    '\nBest Test Score: ', search_space.best_score_,
    '\nHold Out Test Score: ', roc_auc_score(y_test, y_pred_test)
)

In [ ]:
N_ITER = 54
N_JOBS = 1
K_FOLDS = 10
N_ITER_NO_CHANGE = 100
SCORING_METRIC = 'roc_auc'

hyper_parameters = {
        'model__n_estimators': randint(50,500),
        'model__learning_rate': loguniform(3e-4, 3e-1),
        'model__max_depth': randint(3,10),
        'model__max_features': ['sqrt'], 
        'model__min_samples_split': randint(2,20),
        'model__n_iter_no_change': [N_ITER_NO_CHANGE], # Note single elements should still be in a list
        'model__random_state': [RANDOM_STATE],
        'model__min_samples_leaf': randint(1,10)
}

pipe = Pipeline(steps=[
    ('model', GradientBoostingClassifier())
])

search_space = RandomizedSearchCV(
    pipe, hyper_parameters, 
    n_iter = N_ITER, cv=K_FOLDS, 
    scoring=SCORING_METRIC, n_jobs = N_JOBS, 
    return_train_score=True, verbose = 1
)
search_space.fit(X_train, y_train) 

y_pred = search_space.best_estimator_.predict(X_train)
y_pred_prob = search_space.best_estimator_.predict_proba(X_train)[:,1]
y_pred_test = search_space.best_estimator_.predict_proba(X_test)[:,1]

print( 
    'Best Training Score: ', search_space.cv_results_['mean_train_score'][search_space.best_index_], 
    '\nBest Test Score: ', search_space.best_score_,
    '\nHold Out Test Score: ', roc_auc_score(y_test, y_pred_test)
)

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

N_ITER = 54
N_JOBS = 1
hyper_parameters = {
        'n_estimators': Integer(50, 300),
        'max_depth': Integer(3, 10),
        'min_samples_split': Integer(2, 300),
        'learning_rate': Real(3e-6, 3e-1,prior='log-uniform'),     
        'max_features': Categorical(['sqrt', None]),
}
search_space = BayesSearchCV(
    estimator=GradientBoostingClassifier(
        n_iter_no_change=N_ITER_NO_CHANGE, 
        random_state=RANDOM_STATE
    ),
    search_spaces=hyper_parameters,
    scoring=SCORING_METRIC,
    cv=K_FOLDS,
    n_iter=N_ITER,
    random_state=RANDOM_STATE,
    return_train_score=True,
    verbose = 1,
    n_jobs=N_JOBS
)
# executes bayesian optimization
_ = search_space.fit(X_train, y_train)

y_pred = search_space.best_estimator_.predict(X_train)
y_pred_prob = search_space.best_estimator_.predict_proba(X_train)[:,1]
y_pred_test = search_space.best_estimator_.predict_proba(X_test)[:,1]

# model can be saved, used for predictions or scoring
print( 
    'Best Training Score: ', search_space.cv_results_['mean_train_score'][search_space.best_index_], 
    '\nBest Test Score: ', search_space.best_score_,
    '\nHold Out Test Score: ', roc_auc_score(y_test, y_pred_test)
)

In [ ]:
import matplotlib.pyplot as plt
from skopt.plots import plot_objective, plot_histogram

_ = plot_objective(search_space.optimizer_results_[0],
                   n_minimum_search=int(1e8))
plt.show()